In [0]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import re

font_name = fm.FontProperties( fname = '../7주차/NLP/malgun.ttf' ).get_name()
print( font_name )
matplotlib.rc( 'font' , family = font_name )

pd.set_option( 'display.max_rows' , 10000 )
pd.set_option( 'display.max_columns' , 10000 )
pd.set_option( 'display.width' , 10000 )

Malgun Gothic


## 파일 불러오기

In [0]:
dbname = 'database/insta_comment.db'
sql = 'SELECT * FROM split_hashtag'

with sqlite3.connect(dbname) as connection:
    cur = connection.cursor()
    raw_data = cur.execute(sql).fetchall()
    raw_data = pd.DataFrame(raw_data )
    
select_data = raw_data[[0,1,2,3,6,7]]

column_names = ['idx_text' , 'user_id' , 'date' , 'comments' , 'like' , 'keyword']
select_data = pd.DataFrame(select_data.values , columns = column_names)
raw_data = select_data.copy()

## 광고성글 제거하기

In [0]:
import time
# # ['7361412120','23044404754','8512094273','6676042994','2278759108','3175636431','1446151047','5411851063','5531172020','2201236007','7162103099','254913460','1428703068','1186990437']
fake_id = [7361412120,23044404754,8512094273,6676042994,2278759108,3175636431,1446151047,5411851063,5531172020,2201236007,7162103099,254913460,1428703068,1186990437]
id_ = raw_data['user_id'].unique()

fake_id = set(fake_id)
print(len(fake_id))
id_ = set(id_)
print(len(id_))

pure_id = id_ - fake_id
print(len(pure_id))

copy_data = raw_data.copy()
copy_data['pure_user'] = copy_data['user_id'].isin(pure_id)
data_r_fake_id = copy_data.loc[copy_data['pure_user'] == True ]
data = data_r_fake_id.copy()

14
349487
349473


## 코멘트만 가져오기

In [0]:
import re

def get_hash(data):
    reg = re.compile('[\#][ㄱ-힇a-zA-Z0-9]+')
    try:
        res = reg.findall(data)
        txt = ''
        for i in res:
            txt += i
            txt += ' '
        res = txt
    except:
        res = ''
        
    return res
        
def get_comments( data ):
    reg = re.compile('[\#][ㄱ-힇a-zA-Z0-9]+')
    
    res = reg.sub('', data).strip()
      
#     except:
#         res = ''
    
        
    return res
data['pure_comment'] = data['comments'].apply(get_comments)
data['hash'] = data['comments'].apply(get_hash)
output_data = data.loc[data['pure_comment'] != '' ]
output_data.reset_index(drop=True , inplace = True)

## 키워드 색인하기

In [0]:
keyword = raw_data['keyword'].unique() 

keyword
# '라이스버거', '허니버터', '과일소주', '맛스타그램', '시카고피자', '쉑쉑', '방어', '콩불', '훠궈',
#  '감바스', '핫도그', '마라롱샤', '곤약젤리', '마라탕', '탄탄면', '마약김밥', '밥', '모찌'

array(['라이스버거', '슈니발렌', '허니버터', '과일소주', '푸파', '푸드파이터', '맛스타그램', '먹스타그램',
       '시카고피자', '쉑쉑', '딸기모찌', '방어', '팟타이', '콩불', '훠궈', '후쿠오카함바그', '감바스',
       '핫도그', '프랑스가정식', '미국가정식', '푸드스타그램', '마라롱샤', '일본가정식', '대만카스테라',
       '곤약젤리', '펑리수', '마라탕', '나시고랭', '불닭', '냉동삼겹살', '푸드트럭', '탄탄면', '마약김밥',
       '밥', '말차', '모찌'], dtype=object)

In [0]:
food_F = data.loc[(data['keyword'] == '맛스타그램') | (data['keyword'] == '푸드스타그램')|(data['keyword'] == '푸파')|(data['keyword'] == '먹스타그램')]
output_data = food_F[['user_id','date' , 'pure_comment' , 'hash', 'keyword']]
output_data.reset_index(drop = True , inplace = True)

### tokenize화 하기 특정 코멘트

In [0]:
from konlpy.tag import *
# h = Hannanum()
okt = Okt() # okt 사용
# mecab = Mecab()
# ko = Komoran()

def token_han(text):
    token_ = h.pos(text)
    return token_


def token_okt(text):
    token_ = okt.pos(text)
    return token_


def token_mecab(text):
    token_ = mecab.pos(text)
    return token_

def token_komoran(text):
    token_ = ko.pos(text)
    return token_


def token_okt(text):
    token_ = okt.pos(text)
    return token_

def token_okt_selected(text):
    token_ = okt.pos(text)
    res = []
    
    for word , pos in token_:
        if pos in ['Noun' , 'Adjective' , 'Verb' , 'Adverb']:
            res.append(word)
        
    return res

In [0]:
s_data = output_data[['date','pure_comment','hash','keyword']]
s_data = s_data.loc[ s_data['pure_comment'] != '' ].reset_index(drop = True )

def token_okt(text):
    token_ = okt.pos(text)
    return token_

sample_token = s_data.sample(frac = 0.01 , random_state = 3)

sample_token['tokenize'] = sample_token['pure_comment'].apply(token_okt)

## 토큰화 하고 특정 형태소 찾기

In [0]:

    
    for word , pos in token_:
        if pos in ['Noun' , 'Adjective' , 'Verb' , 'Adverb']:
            res.append(word)
        
    return res

def comment_selected(text):
    token_ = okt.pos(text)
    res = ''
    
    for word , pos in token_:
        if pos in ['Noun' , 'Adjective' , 'Verb' , 'Adverb']:
            word = word + ' '
            res += word
        
    return res

sample_token['tokenize_selected'] = sample_token['pure_comment'].apply(token_okt_selected)
sample_token['selected_comment'] = sample_token['pure_comment'].apply(comment_selected)

/Users/joonam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/joonam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## 해시태그 tokenize

In [0]:
badword=['맛스타그램','먹스타그램','푸파','푸드파이터','foodfighter','맛스타그램' ,'일상','데일리','핌스타그램','인스타그램'
        ,'foodstagram','nice','선팔','맞팔','먹스타','먹스타 ','맛스타 ','맛스타','hotelrestaurant','foodstagram ','식신'
         ,'먹스타그램' , '맛스타그램', '데일리' ,'맞팔','선팔','좋아요','진짜','냠냠', '대존맛',
            '맛집추천','음식','너무','술스타그램','koreanfood','럽스타그램','여행스타그램',
            '요리스타그램','좋아요반사','ㅋㅋ','일상스타그램','food','첫줄','f4f',
            '함께','사진','일상','맛집','일상스타그램','푸드스타그램','감사합니다','daily',
            'ㅎㅎ','먹방','존맛','stagram','반사','어제','입니다','어제','카페','있는',
            '스타그램','소통','같이','오늘도','이제','오늘은','소통해요','그리고','인친','follow',
            '소통팔로우','맛은','ㅋㅋㅋ','ㅋㅋㅋㅋ','#오랜만에','주말','l4l','instagood','ootd','맛스타','fff','아주','먹고','먹어','해요',
            '셀카','그냥','ie','셀피','selfie','#얼스타그램','셀스타그램','인스타푸드','얼','like4like','투어','porn','foodporn','좋반',
            '먹으니','해서','하면','ㅠㅠ','likeforlike','먹스타먹스타','환영','다들','안녕','갑자기','insta','그램','gram']
badword.sort(key=len, reverse=True)
badword

def nice_hash(text):
    text = text.replace('#','')
    for bw in badword:
        if bw in text:
            text = text.replace(bw ,'')
            
    return text

sample_token['nice_hash'] = sample_token['hash'].apply(nice_hash)

/Users/joonam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
final_data = sample_data['selected_comment' , 'nice_hash' , 'keyword']

In [0]:
final_data.to_csv('preprocessed_data.csv')